In [1]:
from random import randint as ri, choice as ch

class Ship:
    def __init__(self, length, tp=1, x=None, y=None):
        self._length = length
        self._tp = tp
        self._x = x
        self._y = y
        self._cells = [1 for _ in range(length)]
        self._is_move = True
        self._cells_kort = []

    def set_start_coords(self, x, y):
        self._x = x
        self._y = y

    def get_start_coords(self):
        return self._x, self._y
    
    def set_cells_kort(self):
        x, y, l, t = *self.get_start_coords(), self._length, self._tp
        lst = []
        if t == 2:
            for i in range(l):
                k = (x, y + i)
                lst.append(k)
        elif t == 1:
            for i in range(l):
                k = (x + i, y)
                lst.append(k)
        self._cells_kort = lst
        return lst
    
    def move(self, go):
        if self._is_move:
            if self._tp == 1:
                self._x += go
            elif self._tp == 2:
                self._y += go
                
    def is_collide(self, ship):
        lst, lst1 = [], []
        x, y, l, t = *self.get_start_coords(), self._length, self._tp
        if t == 2:
            for i in range(l + 2):
                for j in range(3):
                    k = (j + x - 1, i + y - 1)
                    lst.append(k)
        elif t == 1:
            for i in range(l + 2):
                for j in range(3):
                    k = (i + x - 1, j + y - 1)
                    lst.append(k)
        lst1 = ship.set_cells_kort()
        for i in lst:
            if i in lst1:
                return True
        return False

    
    def is_out_pole(self, size):
        pole = [[0 for i in range(size)] for j in range(size)]
        x, y = self.get_start_coords()
        if x < 0 or x > size - 1 or y < 0 or y > size - 1:
            return True
        try:
            if self._tp == 1:
                for i in range(size):
                    for j in range(size):
                        if i == x and j == y:
                            for _ in range(self._length):
                                pole[i][j] = 1
                                i += 1
            elif self._tp == 2:
                for i in range(size):
                    for j in range(size):
                        if i == x and j == y:
                            for _ in range(self._length):
                                pole[i][j] = 1
                                j += 1
            return False
        except:
            return True

    def __getitem__(self, indx):
        return self._cells[indx]
    
    def __setitem__(self, key, value):
        self._cells[key] = value


class GamePole:
    COMP = False

    def __init__(self, size):
        self._size = size
        self._ships = []
        self.pole = [[0 for i in range(self._size)] for j in range(self._size)]
        self._crt = []

    def create_ship(self, ship):
        x, y = ship.get_start_coords()
        if ship._tp == 1:
            for i in range(self._size):
                for j in range(self._size):
                    if i == x and j == y:
                        for _ in range(ship._length):
                            self.pole[j][i] = 1
                            ship._cells_kort.append((i, j))
                            self._crt.append((i, j))
                            i += 1
        elif ship._tp == 2:
            for i in range(self._size):
                for j in range(self._size):
                    if i == x and j == y:
                        for _ in range(ship._length):
                            self.pole[j][i] = 1
                            ship._cells_kort.append((i, j))
                            self._crt.append((i, j))
                            j += 1

    def init(self): 
        self.lst_ships = [4, 3, 3, 2, 2, 2, 1, 1, 1, 1]
        for i in self.lst_ships:
            fl = False
            while fl != True:
                ship = Ship(i, tp=ri(1, 2))
                ship.set_start_coords(ri(0, self._size - 1), ri(0, self._size - 1))
                collide = any([ship.is_collide(i) for i in self._ships])
                flk = ship.is_out_pole(self._size)
                a = flk == False
                b = collide == False
                fl = a and b
            self.create_ship(ship)
            self._ships.append(ship)
    
    def get_ships(self):
        return self._ships
    
    def move_ships(self):
        for n in self._ships:
            ls = self._ships[:]
            ls.remove(n)
            go = ch((-1, 1))
            a, b, c, d = n._x, n._y, n._length, n._tp
            if go == 1:
                if d == 1:
                    m = Ship(1, d, a + c, b)
                elif d == 2:
                    m = Ship(1, d, a, b + c)
                collide = any([m.is_collide(i) for i in ls])
                fl = m.is_out_pole(self._size) or collide
                if fl:
                    if d == 1:
                        m = Ship(1, d, a - 1, b)
                    elif d == 2:
                        m = Ship(1, d, a, b - 1)
                    collide = any([m.is_collide(i) for i in ls])
                    fl = m.is_out_pole(self._size) or collide
            else:
                if d == 1:
                    m = Ship(1, d, a - 1, b)
                elif d == 2:
                    m = Ship(1, d, a, b - 1)
                collide = any([m.is_collide(i) for i in ls])
                fl = m.is_out_pole(self._size) or collide
                if fl:
                    if d == 1:
                        m = Ship(1, d, a + c, b)
                    elif d == 2:
                        m = Ship(1, d, a, b + c)
                    collide = any([m.is_collide(i) for i in ls])
                    fl = m.is_out_pole(self._size) or collide
            if not fl and n._is_move:
                for i in n._cells_kort:
                    self.pole[i[1]][i[0]] = 0
                if d == 1:
                    if m._x > a:
                        n.set_start_coords(a + 1, b)
                        n.set_cells_kort()
                    else:
                        n.set_start_coords(a - 1, b)
                        n.set_cells_kort()
                elif d == 2:
                    if m._y > b:
                        n.set_start_coords(a, b + 1)
                        n.set_cells_kort()
                    else:
                        n.set_start_coords(a, b - 1)
                        n.set_cells_kort()
                for i in n._cells_kort:
                    self.pole[i[1]][i[0]] = 1
                    
    def show(self):  
        for i in self.pole:
            for j in i:
                if not self.COMP:
                    print(j, end=' ')
                else:
                    if j in (0, 1, 2):
                        print('.', end=' ')
                    else:
                        print(j, end=' ')
            print()

    def get_pole(self):
        return tuple(tuple(i) for i in self.pole)

    def bomb(self, x, y):
        fl = True
        for i in self._ships:
            if (x, y) in i._cells_kort:
                i._is_move = False
                n = i._cells_kort.index((x, y))
                i._cells[n] = 2
                self.pole[y][x] = 'x'
                if not 1 in i._cells:
                    self._ships.remove(i)
                print(f'Бомба попала в корабль')
                fl = False
                break
        if fl:
            if self.pole[y][x] == 'x':
                print(f'Бомба уже попадала сюда')
            else:
                self.pole[y][x] = '*'
                print(f'Промах')

class SeaBattle:
    def __init__(self, size):
        self.comp = GamePole(size)
        self.Human = GamePole(size)
        self.comp.init()
        self.Human.init()
        self.comp.COMP = True

    def show(self): 
        print("Ваше поле:") 
        self.Human.show()
        print()
        print("Поле компьютера:")
        self.comp.show()

    def game(self):
        print("Подбросим монету на право первого выстрела\nОрел - стреляете первым Вы, Решка - компьютер\nНажмите Enter")
        input()
        self.show()
        a = ch(('Орел', 'Решка'))
        if a == 'Орел':
            print("Орел, введите координаты выстрела - цифры от 0 до 9 через пробел\nНажмите Enter\nКомпьютер будет стрелять сразу после вас")
            x, y = map(int, input().split())
            self.comp.bomb(x, y)
        else:
            print("Решка, компьютер уже выстрелил\nДальше будет стрелять сразу после вас")
            self.Human.bomb(ri(0, self.comp._size - 1), ri(0, self.comp._size - 1))
            self.show()
            print("Ваш выстрел")
            x, y = map(int, input().split())
            self.comp.bomb(x, y)
        self.Human.bomb(ri(0, self.comp._size - 1), ri(0, self.comp._size - 1))
        self.show()
        while True:
            print("Ваш выстрел")
            x, y = map(int, input().split())
            self.comp.bomb(x, y)
            self.Human.bomb(ri(0, self.comp._size - 1), ri(0, self.comp._size - 1))
            self.comp.move_ships()
            self.Human.move_ships()
            self.show()
            if self.Human._ships == []:
                print("Компьютер победил!")
                break
            if self.comp._ships == []:
                print("Вы победили!")
                break
        self.show()


Battle = SeaBattle(10)
Battle.game()
